# Epicollect

In [57]:
import pandas as pd
import requests
import json
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
requests.packages.urllib3.disable_warnings()

In [58]:
def gera_token(CLIENT_ID, CLIENT_SECRET):
    """
    Função gera token de acesso ao aplicativo
        CLIENT_ID: identificador do cliente
        CLIENT_SECRET: chave de acesso do cliente
    """
    url = 'https://five.epicollect.net/api/oauth/token'
    dados = {'grant_type': 'client_credentials', 'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET}
    resposta = requests.post(url, data=dados, verify=False)
    return resposta.json()

In [59]:
def gera_dataframe(token, SLUG, origem, registros_por_pagina):
    """
    Função para obter registros dos form e branch
        token: token de acesso ao aplicativo
        SLUG: versão slug do aplicativo
        origem: define se será o form ou branch
        registros_por_pagina: define o número máximo de registros a serem recuperados por consulta
    """
    df = pd.DataFrame()
    url = f"https://five.epicollect.net/api/export/entries/{SLUG}?map_index=0&{origem}&format=json&headers=true&per_page={registros_por_pagina}"
    registros=(requests.get(url, headers={"Authorization": f"Bearer {token['access_token']}"}, verify=False)).json()
    ultima_pagina = registros['meta']['last_page']
    df = pd.concat([df,pd.DataFrame.from_dict(registros['data']['entries'])], axis=0)
    if ultima_pagina>1:
        for registro in range(2,ultima_pagina+1):
            registros=(requests.get(url, headers={"Authorization": f"Bearer {token['access_token']}"}, verify=False)).json()
            df = pd.concat([df,pd.DataFrame.from_dict(registros['data']['entries'])], axis=0)
    return df

In [60]:
def dados(propriedades, amostras):
    """
    Função realiza a união dos registros de propriedade e amostras
        propriedades: dataframe com informações de propriedades
        amostras: dataframe com informações de amostras
    """
    propriedades.set_index('ec5_uuid', inplace=True)
    amostras.set_index('ec5_branch_owner_uuid', inplace=True)
    return propriedades.join(amostras, how='outer',rsuffix='_amostra')

## Definição dos parâmetros do projeto
Estes parâmetros vêm do epicollect ao clicar na API serão apresentados

In [61]:
CLIENT_ID = 
CLIENT_SECRET = ''
NAME = ''
SLUG = ''
form_ref=''
branch_ref=''
registros_por_pagina = 1000 #Máximo que epicollect permite

# Dados dos arquivos de saída
nome_arquivo_geral = 'dados_epicollect.xlsx'
nome_arquivo_propriedades = 'propriedades.xlsx'
nome_arquivo_amostras = 'amostras.xlsx'

Não foi possível utilizar o pacote pyepicollect. Existe um proxy e a utilização do pacote gera um erro ralacionado ao certificado. Optou-se por escrever o código do zero.

## Obtendo um token de acesso (válido por duas horas)

In [62]:
token = gera_token(CLIENT_ID, CLIENT_SECRET)

## Obtendo dados do formulário principal e gerando colunas com coordenadas

In [63]:
origem = f'form_ref={form_ref}'
propriedades = gera_dataframe(token, SLUG, origem, registros_por_pagina)
propriedades['latitude'] = propriedades['12_Geolocalizao'].apply(lambda x: x['latitude'])
propriedades['longitude'] = propriedades['12_Geolocalizao'].apply(lambda x: x['longitude'])
propriedades['8_Quais_espcies_pres'] = propriedades['8_Quais_espcies_pres'].apply(lambda x: ', '.join(x))
propriedades['20_Risco_atribudo_ao'] = propriedades['20_Risco_atribudo_ao'].apply(lambda x: ', '.join(x))
propriedades.drop('12_Geolocalizao', axis=1, inplace=True)

## Obtendo dados do formulário acessório

In [64]:
origem = f'branch_ref={branch_ref}'
amostras = gera_dataframe(token, SLUG, origem, registros_por_pagina)

## Grava planilha excel

In [65]:
planilha = dados(propriedades,amostras)

In [66]:
propriedades.to_excel(nome_arquivo_propriedades)
amostras.to_excel(nome_arquivo_amostras)
planilha.to_excel(nome_arquivo_geral)

## Exporta como Geopackage

In [67]:
import geopandas as gpd

In [68]:
gdf = gpd.GeoDataFrame(propriedades, geometry=gpd.points_from_xy(propriedades.longitude, propriedades.latitude), crs="EPSG:4326")

In [69]:
gdf.to_file('propriedades.gpkg', driver='GPKG', layer='Propriedades')